In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
pd.set_option('display.max_columns', None)

In [40]:
print("Enter your file choice: 1/2/3")
choice=int(input())
if choice==1:
    dataframe=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
#dataframe
#print(dataframe.head(),5)

Enter your file choice: 1/2/3


In [41]:
def preprocessing(ch,df):
    #replace empty string values with NaN
    df = df.replace(" ", np.nan)
    #replace missing values with columnwise mean for numeric columns
    numeric_cols = df.select_dtypes(include='number')
    df[numeric_cols.columns] = numeric_cols.fillna(numeric_cols.mean())
    #replace missing values with columnwise mode for non-numeric columns
    non_numeric_cols = df.select_dtypes(exclude='number')
    df[non_numeric_cols.columns] = non_numeric_cols.fillna(non_numeric_cols.mode().iloc[0])
    #remove duplicate rows
    df.drop_duplicates(inplace=True)

    if ch==1:
        features=dataframe.drop('Churn',axis=1)
        features.drop('customerID',axis=1,inplace=True)
        target=dataframe['Churn']

    #label encoding target column
    encoder=LabelEncoder()
    target=encoder.fit_transform(target)

    #one hot encoding the categorical columns
    categorical_columns=[col for col in features if features[col].dtype == 'object']
    for col in categorical_columns:
        features=pd.get_dummies(features, columns=[col], drop_first=True)

    #min-max scaling the numeric columns
    scaler=MinMaxScaler()
    scale_columns=features.select_dtypes(exclude=['bool']).columns
    features[scale_columns]=scaler.fit_transform(features[scale_columns])

    features_df=pd.DataFrame(features,columns=features.columns)
    target_df=pd.DataFrame(target,columns=['Churn'])
    correlations=features_df.corrwith(target_df['Churn'])

    return features_df,target_df,correlations

In [50]:
feature_processed, target_processed, correlations=preprocessing(choice,dataframe)
norm=Normalizer()
feature_normalized=norm.fit_transform(feature_processed)
#feature_processed

In [51]:
top_20_correlations=correlations.abs().sort_values(ascending=False).head(20)

feature_processed_df=pd.DataFrame(feature_normalized,columns=feature_processed.columns)
feature_processed_df=feature_processed_df[top_20_correlations.index]


In [54]:
X_train, X_test, y_train, y_test = train_test_split(feature_processed_df, target_processed, test_size=0.2, random_state=42)
# Step 2: Initialize the Logistic Regression classifier
clf = LogisticRegression()
# Step 3: Train the classifier on the training data
clf.fit(X_train, y_train)
# Step 4: Make predictions on the test set
y_pred = clf.predict(X_test)
# Step 5: Evaluate the classifier's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of Logistic Regression classifier: {accuracy:.2f}")


Accuracy of Logistic Regression classifier: 0.81


c:\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Bagging

In [57]:
num_bootstrap_samples=9
bootstrap_samples=[]

X_train_np = X_train.to_numpy()  # Convert X_train to a NumPy array

for i in range(num_bootstrap_samples):
    bootstrap_indices=np.random.choice(range(len(X_train)),len(X_train),replace=True)
    bootstrap_X=X_train_np[bootstrap_indices]
    bootstrap_y=y_train.iloc[bootstrap_indices]
    bootstrap_samples.append((bootstrap_X, bootstrap_y))

lr_models=[]
for x,y in bootstrap_samples:
    clf=LogisticRegression()
    clf.fit(x,y)
    lr_models.append(clf)

predictions=[]
for model in lr_models:
    predictions.append(model.predict(X_test))
predictions=np.array(predictions)
aggregated_predictions=np.mean(predictions,axis=0)

# Round the aggregated predictions to the nearest integer
aggregated_predictions = np.round(aggregated_predictions)

bagging_accuracy=accuracy_score(y_test,aggregated_predictions)
print(f"Accuracy of Bagging classifier: {bagging_accuracy:.2f}")

Accuracy of Bagging classifier: 0.81


c:\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Python310\lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1